In [1]:
import org.json4s._
import org.json4s.jackson.JsonMethods._
import scala.runtime.ScalaRunTime._

Intitializing Scala interpreter ...

Spark Web UI available at http://LAPTOP-PC8F3R05:4041
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1587886592235)
SparkSession available as 'spark'


import org.json4s._
import org.json4s.jackson.JsonMethods._
import scala.runtime.ScalaRunTime._


In [2]:
val file_path = "D:\\TU WIEN\\DIC SS20\\Assignment_2\\reviews_devset.json"
//val peopleDF = spark.read.json(path)
//peopleDF.printSchema()

file_path: String = D:\TU WIEN\DIC SS20\Assignment_2\reviews_devset.json


# Names of the columns:

root <br>
 |-- asin: string (nullable = true) <br>
 |-- category: string (nullable = true) <br>
 |-- helpful: array (nullable = true) <br>
 |    |-- element: long (containsNull = true) <br>
 |-- overall: double (nullable = true) <br>
 |-- reviewText: string (nullable = true) <br>
 |-- reviewTime: string (nullable = true) <br>
 |-- reviewerID: string (nullable = true) <br>
 |-- reviewerName: string (nullable = true) <br>
 |-- summary: string (nullable = true) <br>
 |-- unixReviewTime: long (nullable = true) <br>

In [3]:
val rdd = sc.textFile(file_path)

/* Works:
rdd.map{ row =>
  val json_row = parse(row)

  (compact(json_row \ "asin") + "test", compact(json_row \ "reviewText"))
}.collect().foreach{println _}
*/

rdd: org.apache.spark.rdd.RDD[String] = D:\TU WIEN\DIC SS20\Assignment_2\reviews_devset.json MapPartitionsRDD[1] at textFile at <console>:36


In [4]:
val rdd_text = rdd.map{ row =>
      val json_row = parse(row)

      (compact(json_row \ "reviewText"), compact(json_row \ "category"), compact(json_row \ "asin") + compact(json_row \ "reviewerID"))
    }

rdd_text: org.apache.spark.rdd.RDD[(String, String, String)] = MapPartitionsRDD[2] at map at <console>:35


In [5]:
/* With array 
val rdd_text_split = rdd_text.map(x => x.split(Array('.' , ' ', '?', '!', ',', ';', ':', '(', ')', '[',']','{','}','-','_',
                                                    '`','~','#','&','*','%','$')))
*/
val rdd_text_split = rdd_text.map(x =>(
"\\s+|\\,|\\\\|/|\\.|\\!|\\?|\\;|\\:|\\(|\\)|\\[|\\]|\\{|\\}|\\-|\\_|\"|\\`|\\~|\\#|\\&|\\*|\\%|\\$".r.split(x._1).filter(y => y.length > 1), x._2, x._3))

rdd_text_split.take(10).foreach(array => println(stringOf(array)))

(Array(This, was, gift, for, my, other, husband, He's, making, us, things, from, it, all, the, time, and, we, love, the, food, Directions, are, simple, easy, to, read, and, interpret, and, fun, to, make, We, all, love, different, kinds, of, cuisine, and, Raichlen, provides, recipes, from, everywhere, along, the, barbecue, trail, as, he, calls, it, Get, it, and, just, open, page, Have, at, it, You'll, love, the, food, and, it, has, provided, us, with, an, insight, into, the, culture, that, produced, it, It's, all, about, broadening, horizons, Yum),"Patio_Lawn_and_Garde","0981850006""A2VNYWOPJ13AFP")
(Array(This, is, very, nice, spreader, It, feels, very, solid, and, the, pneumatic, tires, give, it, great, maneuverability, and, handling, over, bumps, The, control, arm, is, solid, metal, not, cable, which, gives, you, precise, control, and, will, last, long, time, The, settings, take, some, experimentation, with, your, various, products, to, get, it, right, but, that, is, true, of, any, s

rdd_text_split: org.apache.spark.rdd.RDD[(Array[String], String, String)] = MapPartitionsRDD[3] at map at <console>:39


In [6]:
val rdd_text_no_duplicates = rdd_text_split.map(x => (x._1.toSet.toList, x._2,x._3))

rdd_text_no_duplicates: org.apache.spark.rdd.RDD[(List[String], String, String)] = MapPartitionsRDD[4] at map at <console>:35


In [7]:
val rdd_text_casefolding = rdd_text_no_duplicates.map(x => (x._1.map(_.toLowerCase()),x._2,x._3))
rdd_text_casefolding.take(2).foreach(array => println(stringOf(array)))

(List(barbecue, read, broadening, for, recipes, raichlen, insight, provided, are, it's, you'll, produced, this, different, get, all, he's, trail, page, gift, horizons, just, us, it, calls, provides, as, have, has, we, husband, open, culture, cuisine, that, to, making, was, at, kinds, my, directions, everywhere, easy, food, simple, things, he, love, with, from, fun, make, an, into, time, about, we, along, yum, other, of, and, interpret, the),"Patio_Lawn_and_Garde","0981850006""A2VNYWOPJ13AFP")
(List(side, maneuverability, any, your, is, than, experimentation, solid, this, but, good, distribution, handling, pneumatic, left, it, cable, has, crappy, nice, products, feels, far, last, flings, that, arm, to, you, probably, various, was, long, gives, the, over, on, my, precise, take, spreader, farther, even, will, little, not, with, give, edgeguard, true, it, which, control, material, get, time, more, metal, very, some, great, of, settings, and, right, ever, tires, bumps, the),"Patio_Lawn_and_

rdd_text_casefolding: org.apache.spark.rdd.RDD[(List[String], String, String)] = MapPartitionsRDD[5] at map at <console>:35


In [8]:
//creating stopWords List:
import scala.io.Source
import scala.collection.mutable.ListBuffer

var stopWordsHelper = new ListBuffer[String]()
val filename = "stopwords.txt"
for (line <- Source.fromFile(filename).getLines) {
    stopWordsHelper += line
}
val stopWords = stopWordsHelper.toList

import scala.io.Source
import scala.collection.mutable.ListBuffer
stopWordsHelper: scala.collection.mutable.ListBuffer[String] = ListBuffer(a's, able, about, above, according, accordingly, across, actually, after, afterwards, again, against, ain't, all, allow, allows, almost, alone, along, already, also, although, always, am, among, amongst, an, and, another, any, anybody, anyhow, anyone, anything, anyway, anyways, anywhere, apart, appear, appreciate, appropriate, are, aren't, around, as, aside, ask, asking, associated, at, available, away, awfully, be, became, because, become, becomes, becoming, been, before, beforehand, behind, being, believe, below, beside, besides, best, better, between, beyond, both, brief, but, by, c'mon, c's, came, can, can't, cannot, cant, cause, causes, certa...

In [9]:
val rdd_text_no_stopwords = rdd_text_casefolding.map(x => (x._1.filter(!stopWords.contains(_)),x._2,x._3))
rdd_text_no_stopwords.take(2).foreach(array => println(stringOf(array)))

(List(barbecue, read, broadening, recipes, raichlen, insight, provided, produced, trail, page, gift, horizons, calls, husband, open, culture, cuisine, making, kinds, directions, easy, food, simple, things, love, fun, make, time, yum, interpret),"Patio_Lawn_and_Garde","0981850006""A2VNYWOPJ13AFP")
(List(side, maneuverability, experimentation, solid, good, distribution, handling, pneumatic, left, cable, crappy, nice, products, feels, flings, arm, long, precise, spreader, farther, give, edgeguard, true, control, material, time, metal, great, settings, tires, bumps),"Patio_Lawn_and_Garde","B00002N66D""A2E5XXXC07AGA7")


rdd_text_no_stopwords: org.apache.spark.rdd.RDD[(List[String], String, String)] = MapPartitionsRDD[6] at map at <console>:39


 # Get the amount of documents per category

In [10]:
val category_count = rdd.map{ row =>
  val json_row = parse(row)
  (compact(json_row \ "category").replace("\"", ""), 1)
}.reduceByKey((x,y) => x+y)

category_count: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[8] at reduceByKey at <console>:40


In [11]:
category_count.collect().foreach(println)

(Pet_Supplie,1235)
(Movies_and_TV,4607)
(Electronic,7825)
(Office_Product,1243)
(Musical_Instrument,500)
(Cell_Phones_and_Accessorie,3447)
(Clothing_Shoes_and_Jewelry,5749)
(Tools_and_Home_Improvement,1926)
(Baby,916)
(Patio_Lawn_and_Garde,994)
(Home_and_Kitche,4254)
(Sports_and_Outdoor,3269)
(Toys_and_Game,2253)
(Digital_Music,836)
(Apps_for_Android,2638)
(Grocery_and_Gourmet_Food,1297)
(Beauty,2023)
(Health_and_Personal_Care,2982)
(Book,22507)
(Automotive,1374)
(Kindle_Store,3205)
(CDs_and_Vinyl,3749)


In [12]:
val theMap =category_count.collectAsMap

theMap: scala.collection.Map[String,Int] = Map(Kindle_Store -> 3205, Electronic -> 7825, Automotive -> 1374, Pet_Supplie -> 1235, Clothing_Shoes_and_Jewelry -> 5749, Baby -> 916, Grocery_and_Gourmet_Food -> 1297, Musical_Instrument -> 500, Book -> 22507, Movies_and_TV -> 4607, Tools_and_Home_Improvement -> 1926, Sports_and_Outdoor -> 3269, CDs_and_Vinyl -> 3749, Apps_for_Android -> 2638, Home_and_Kitche -> 4254, Office_Product -> 1243, Health_and_Personal_Care -> 2982, Digital_Music -> 836, Cell_Phones_and_Accessorie -> 3447, Beauty -> 2023, Toys_and_Game -> 2253, Patio_Lawn_and_Garde -> 994)


In [13]:
 theMap.keys.foreach{ i =>  
 print( "Key = " + i )
 println(" Value = " + theMap(i) )}

Key = Kindle_Store Value = 3205
Key = Electronic Value = 7825
Key = Automotive Value = 1374
Key = Pet_Supplie Value = 1235
Key = Clothing_Shoes_and_Jewelry Value = 5749
Key = Baby Value = 916
Key = Grocery_and_Gourmet_Food Value = 1297
Key = Musical_Instrument Value = 500
Key = Book Value = 22507
Key = Movies_and_TV Value = 4607
Key = Tools_and_Home_Improvement Value = 1926
Key = Sports_and_Outdoor Value = 3269
Key = CDs_and_Vinyl Value = 3749
Key = Apps_for_Android Value = 2638
Key = Home_and_Kitche Value = 4254
Key = Office_Product Value = 1243
Key = Health_and_Personal_Care Value = 2982
Key = Digital_Music Value = 836
Key = Cell_Phones_and_Accessorie Value = 3447
Key = Beauty Value = 2023
Key = Toys_and_Game Value = 2253
Key = Patio_Lawn_and_Garde Value = 994


In [14]:
theMap("Kindle_Store")

res5: Int = 3205


# Calculating ChiSquare Values

Idea: create rdd tuples like this: ((term, category), 1)
then, reduce by key by summing up the values.

In [15]:
//main idea: https://stackoverflow.com/questions/39005801/mapvalues-and-explode-in-rdd
val convert: ((List[String], String)) => List[((String, String), Int)] = tuple => tuple._1.map(term =>
  ((term, tuple._2.replace("\"", "")), 1)).toList

val chiSq1 = rdd_text_no_stopwords.map(x => (x._1, x._2)).flatMap(convert).reduceByKey((x,y) => x + y)
chiSq1.take(20).foreach(println)

((mezzos,Movies_and_TV),1)
((huge,Cell_Phones_and_Accessorie),36)
((goodwill,Electronic),2)
((lit,Tools_and_Home_Improvement),13)
((alittle,Health_and_Personal_Care),2)
((aspca,Pet_Supplie),1)
((alwasy,Book),1)
((scales,Digital_Music),1)
((instructor,Movies_and_TV),11)
((upright,Automotive),2)
((personally,Apps_for_Android),5)
((absorb,Baby),4)
((repopulate,Book),1)
((perfect,Movies_and_TV),194)
((rib,Kindle_Store),1)
((tubular,Health_and_Personal_Care),1)
((ten,Patio_Lawn_and_Garde),4)
((duet,Musical_Instrument),2)
((stripped,Electronic),5)
((trevor's,Book),1)


convert: ((List[String], String)) => List[((String, String), Int)] = <function1>
chiSq1: org.apache.spark.rdd.RDD[((String, String), Int)] = ShuffledRDD[11] at reduceByKey at <console>:41


In [16]:
val chiSq2 = chiSq1.map(x => (x._1._1, (x._1._2, x._2)))
chiSq2.take(5).foreach(println)

(mezzos,(Movies_and_TV,1))
(huge,(Cell_Phones_and_Accessorie,36))
(goodwill,(Electronic,2))
(lit,(Tools_and_Home_Improvement,13))
(alittle,(Health_and_Personal_Care,2))


chiSq2: org.apache.spark.rdd.RDD[(String, (String, Int))] = MapPartitionsRDD[12] at map at <console>:37


In [17]:
val chiSq3 = chiSq2.groupByKey()
chiSq3.take(2).foreach(println)

(vecindad,CompactBuffer((Movies_and_TV,1)))
(bone,CompactBuffer((Pet_Supplie,9), (Home_and_Kitche,3), (Cell_Phones_and_Accessorie,3), (Sports_and_Outdoor,5), (Toys_and_Game,1), (CDs_and_Vinyl,11), (Book,26), (Tools_and_Home_Improvement,1), (Digital_Music,1), (Automotive,1), (Kindle_Store,2), (Beauty,2), (Musical_Instrument,1), (Patio_Lawn_and_Garde,1), (Movies_and_TV,9), (Clothing_Shoes_and_Jewelry,4), (Health_and_Personal_Care,13)))


chiSq3: org.apache.spark.rdd.RDD[(String, Iterable[(String, Int)])] = ShuffledRDD[13] at groupByKey at <console>:37


In [18]:
val chiSq4 = chiSq3.map(x => (x._1, x._2.toList)).map(x => (x._1, x._2.groupBy(_._1).map { case (k,v) => (k,v.map(_._2))}))

chiSq4: org.apache.spark.rdd.RDD[(String, scala.collection.immutable.Map[String,List[Int]])] = MapPartitionsRDD[15] at map at <console>:37


In [19]:
chiSq4.take(5).foreach(println)

(vecindad,Map(Movies_and_TV -> List(1)))
(bone,Map(Patio_Lawn_and_Garde -> List(1), Movies_and_TV -> List(9), Tools_and_Home_Improvement -> List(1), Kindle_Store -> List(2), Home_and_Kitche -> List(3), Digital_Music -> List(1), Automotive -> List(1), Book -> List(26), Clothing_Shoes_and_Jewelry -> List(4), Toys_and_Game -> List(1), Health_and_Personal_Care -> List(13), Sports_and_Outdoor -> List(5), Beauty -> List(2), CDs_and_Vinyl -> List(11), Musical_Instrument -> List(1), Cell_Phones_and_Accessorie -> List(3), Pet_Supplie -> List(9)))
(glorifying,Map(Book -> List(4), Movies_and_TV -> List(1)))
(rollon,Map(Beauty -> List(1)))
(folan,Map(Movies_and_TV -> List(1)))


In [256]:
val testlist = ("a", 1) :: ("b", 2) :: Nil
val testdict = testlist

testlist: List[(String, Int)] = List((a,1), (b,2))
testdict: scala.collection.immutable.Map[String,List[Int]] = Map(b -> List(2), a -> List(1))


In [260]:
testdict.keys

res153: Iterable[String] = Set(b, a)


In [30]:
def chiSqlCalc (t: (String, Map[String, List[Int]])): (String, Map[String, List[Int]]) = {
        t
}
val ttuple = ("vecindad",Map("Movies_and_TV" -> List(1)))

chiSqlCalc: (t: (String, Map[String,List[Int]]))(String, Map[String,List[Int]])
ttuple: (String, scala.collection.immutable.Map[String,List[Int]]) = (vecindad,Map(Movies_and_TV -> List(1)))


In [32]:
val res_ttuple = chiSqlCalc(ttuple)

res_ttuple: (String, Map[String,List[Int]]) = (vecindad,Map(Movies_and_TV -> List(1)))


In [34]:
chiSq4.map(x => chiSqlCalc(x)).take(5).foreach(println)

(vecindad,Map(Movies_and_TV -> List(1)))
(bone,Map(Patio_Lawn_and_Garde -> List(1), Movies_and_TV -> List(9), Tools_and_Home_Improvement -> List(1), Kindle_Store -> List(2), Home_and_Kitche -> List(3), Digital_Music -> List(1), Automotive -> List(1), Book -> List(26), Clothing_Shoes_and_Jewelry -> List(4), Toys_and_Game -> List(1), Health_and_Personal_Care -> List(13), Sports_and_Outdoor -> List(5), Beauty -> List(2), CDs_and_Vinyl -> List(11), Musical_Instrument -> List(1), Cell_Phones_and_Accessorie -> List(3), Pet_Supplie -> List(9)))
(glorifying,Map(Book -> List(4), Movies_and_TV -> List(1)))
(rollon,Map(Beauty -> List(1)))
(folan,Map(Movies_and_TV -> List(1)))


In [ ]:
/*
 skeleton function to ChiSq:
 
 def chiSqlCalc (t: (String, Map[String, List[Int]])): (String, Map[String, List[Int]]) = {
        t
}

chiSq4.map(x => chiSqlCalc(x)).take(5).foreach(println)
*/